In [1]:
import sqlite3
from datetime import datetime
from pathlib import Path

import hjson
import pandas as pd
from gf_utils.stc_data import get_stc_data
from event_record import EventRecord

gamedata = get_stc_data(
    ["../GF_Data_Tools/data/ch/stc", "../GF_Data_Tools/data/ch/catchdata"],
    "../GF_Data_Tools/data/ch/asset/table",
)
path = Path("../Elisa/logs/develop_log.db").resolve()
conn = sqlite3.connect(f"file://{path}?immutable=1", uri=True)

In [131]:

eventrecord = EventRecord("./events.hjson")

In [2]:
equip = pd.read_sql("""
select * from equip
""", conn)

In [164]:
def analyze(df):
    series = dict()
    for formula in filter(lambda x: x["develop_type"]==3,gamedata["recommended_formula"].values()):
        mp, ammo, mre, part = formula["produce_mp"], formula["produce_ammo"], formula["produce_mre"], formula["produce_part"]
        sub_df = df.query("mp==@mp and ammo==@ammo and mre==@mre and part==@part")
        if len(sub_df)==0:
            continue
        # print(mp, ammo, mre, part)

        prev = {k: [int(i) for i in v.split(",")] for k,v in map(lambda x: x.split(":"), formula["preview"].split("|"))}
        rarity = {k: v for k,v in map(lambda x: x.split(":"),formula["type_rarity"].split(","))}
        eqs = filter(lambda x: gamedata["equip"][x]["rank"]==5,sum([v for k,v in prev.items() if rarity[k]=="A"], start=[]))
        # print({k: gamedata["equip"][k]["name"] for k in eqs})
        cnt = len(sub_df)
        record = {}
        record["total"] = cnt
        record["rank5"] = sum(sub_df["equip_rank"]==5)/cnt
        for eid in eqs:
            record[gamedata["equip"][eid]["name"]] = sum(sub_df["equip_id"]==eid)/cnt
        for k, v in record.items():
            series[(mp,ammo,mre,part,k)] = v
    series = pd.Series(series)
    series.index.set_names(["mp","ammo","mre","part","..."],inplace=True)
    return series
        
df = equip.query("trust==True and item_num==1")
result = []
for name, query in eventrecord["equip"]:
    sub_df = df.query(query)
    ret = analyze(sub_df)
    ret.name = name
    result.append(ret)

pd.concat(result, axis=1)

日常（脚架）     圣建（23春节）    日常（30200）  \
mp  ammo mre part ...                                                   
140 10   110 10   total        31499.000000  2711.000000  5301.000000   
                  rank5            0.027144     0.042789     0.025844   
                  VFL 6-24X56      0.008699     0.012541     0.006414   
                  EOT 518          0.008349     0.012910     0.006603   
                  ITI MARS         0.007905     0.014017     0.007734   
...                                     ...          ...          ...   
80  220  80  80   rank5            0.045691     0.061911     0.038108   
                  Mk0多用途喉缩管        0.028118     0.034993     0.021025   
100 250  150 170  total        10153.000000  2739.000000  1133.000000   
                  rank5            0.069832     0.075940     0.069726   
                  M200X LRF        0.049050     0.048923     0.051192   

                                  圣建（23周年）  
mp  ammo mre part ...                       
140 10   110 10   total        2688.000000  
                  rank5           0.027902  
                  VFL 6-24X56     0.007440  
                  EOT 518         0.007068  
                  ITI MARS        0.008185  
...                                    ...  
80  220  80  80   rank5           0.058894  
                  Mk0多用途喉缩管       0.036058  
100 250  150 170  total         929.000000  
                  rank5           0.069968  
                  M200X LRF       0.052745  

[61 rows x 4 columns]

In [167]:
a.to_excel("test.xlsx")

In [132]:
import pandas as pd

def analyze_period(df: pd.DataFrame):
    ret = dict(total={}, rank5={}, prob={})
    for formula in filter(lambda x: x["develop_type"]==3,gamedata["recommended_formula"].values()):
        mp, ammo, mre, part = formula["produce_mp"], formula["produce_ammo"], formula["produce_mre"], formula["produce_part"]
        sub_df = df.query("mp==@mp and ammo==@ammo and mre==@mre and part==@part")
        total = len(sub_df)
        cnt = sub_df.value_counts("equip_rank").get(5,0)
        if total>0:
            ret["total"][(mp, ammo, mre, part)] = total
            ret["rank5"][(mp, ammo, mre, part)] = cnt
            ret["prob"][(mp, ammo, mre, part)] = cnt/total
    return ret


# df = equip.query("dev_time > 1684684800 and dev_time < 1684857600 and trust==True and item_num==1")
# analyze_period(df)

In [136]:
df = equip.query("trust==True and item_num==1")

In [137]:
result = {}
for name, query in eventrecord["equip"]:
    sub_df = df.query(query)
    for k, v in analyze_period(sub_df).items():
        result[(name, k)] = v

In [138]:
result_df = pd.DataFrame.from_dict(result)

In [139]:
result_df

日常（脚架）                 圣建（23春节）                 日常（30200）  \
                 total rank5      prob    total rank5      prob     total   
140 10  110 10   31499   855  0.027144     2711   116  0.042789      5301   
70  10  150 30   69938  2122  0.030341    17968   839  0.046694      9390   
150 50  50  50   11252   386  0.034305      902    44  0.048780      1633   
10  150 90  100  15930   678  0.042561     1983   123  0.062027      2237   
180 180 10  50   27890  1144  0.041018     2694   145  0.053823      4566   
10  230 120 80   19638   818  0.041654     1891   124  0.065574      2392   
30  150 30  130  11722   560  0.047773     2270   159  0.070044      3483   
100 80  10  100  28596  1040  0.036369     4205   212  0.050416      4368   
50  50  100 100   5120   216  0.042188      501    30  0.059880       856   
30  30  30  200   7756   381  0.049123      567    37  0.065256      1141   
180 20  200 20   10965   387  0.035294     1279    72  0.056294      1050   
    65  65  65    6239   258  0.041353      315    28  0.088889      1214   
170 120 50  50   29646  1189  0.040107     7077   442  0.062456      4674   
10  300 10  10    9468   299  0.031580      906    41  0.045254      1094   
300 10  10  200   6893   480  0.069636     1307   123  0.094109       698   
80  220 80  80    8251   377  0.045691     2229   138  0.061911       761   
100 250 150 170  10153   709  0.069832     2739   208  0.075940      1133   

                                圣建（23周年）                  
                rank5      prob    total rank5      prob  
140 10  110 10    137  0.025844     2688    75  0.027902  
70  10  150 30    263  0.028009    14863   462  0.031084  
150 50  50  50     64  0.039192      588    23  0.039116  
10  150 90  100   114  0.050961     2928   126  0.043033  
180 180 10  50    181  0.039641     3012   123  0.040837  
10  230 120 80     97  0.040552     1100    59  0.053636  
30  150 30  130   143  0.041057     2789   136  0.048763  
100 80  10  100   185  0.042353     1791    79  0.044109  
50  50  100 100    34  0.039720      538    20  0.037175  
30  30  30  200    57  0.049956      949    48  0.050580  
180 20  200 20     42  0.040000      652    19  0.029141  
    65  65  65     42  0.034596     1454    66  0.045392  
170 120 50  50    203  0.043432     5538   229  0.041351  
10  300 10  10     45  0.041133      600    18  0.030000  
300 10  10  200    40  0.057307     1032    67  0.064922  
80  220 80  80     29  0.038108      832    49  0.058894  
100 250 150 170    79  0.069726      929    65  0.069968

In [109]:
def analyze_equip(df, mp, ammo, mre, part):
    df = df.query("mp==@mp and ammo==@ammo and mre==@mre and part==@part")
    print(len(df))
    cnt  = df.value_counts(["equip_id","equip_rank"], normalize=True)
    # cnt = cnt["equip_rank"]==5
    return cnt[cnt.index.get_level_values("equip_rank")==5]

result = []
for name, query in eventrecord["equip"]:
    sub_df = df.query(query)
    cnt = analyze_equip(sub_df, 10, 10, 10, 10)
    cnt.name = name
    result.append(cnt)
result_df = pd.concat(result, axis=1).fillna(0)



190925
4356
3363


In [106]:
result_df["name"] = list(map(lambda x: gamedata["equip"][x]["name"], result_df.index.get_level_values("equip_id")))

In [117]:
" or ".join(map(lambda x: f"equip_id=={x}",result_df[result_df["圣建（23春节）"]==0].index.get_level_values("equip_id")))

'equip_id==72 or equip_id==227 or equip_id==58 or equip_id==308 or equip_id==20 or equip_id==32 or equip_id==40 or equip_id==311 or equip_id==314 or equip_id==231 or equip_id==229 or equip_id==225 or equip_id==68 or equip_id==80 or equip_id==76 or equip_id==235'

In [119]:
from datetime import datetime
datetime.fromtimestamp(1674992066)

datetime.datetime(2023, 1, 29, 19, 34, 26)

In [128]:
df = equip.query("mp==10 and ammo==10 and mre==10 and part==10")
t = df.groupby("equip_id").min()[["equip_name","dev_time"]].sort_values("dev_time")
t["time"] = t.apply(lambda x: datetime.fromtimestamp(x["dev_time"]), axis=1)

/tmp/ipykernel_36267/1366633719.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  t = df.groupby("equip_id").min()[["equip_name","dev_time"]].sort_values("dev_time")
